In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
from string import punctuation

In [3]:
df = pd.read_csv('Combined_News_DJIA.csv')
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [4]:
train_num = int(df.shape[0]*0.6)
df_train = df.loc[:train_num, ]
df_test = df.loc[train_num+1:, ]

In [5]:
## We will be using LSTM in Keras for this problem
## Also used word embeddings  
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
## For RNNs, trying without stemming and removing stopwords
# from collections import Counter
# from nltk.stem.porter import PorterStemmer
# from nltk.corpus import stopwords
from string import punctuation
# Import data
df = pd.read_csv("Combined_News_DJIA.csv")
names = df.columns.values
### Dividing the data into test and train by dates (as specified)
trainStartDate = '2008-08-08' 
trainEndDate = '2013-08-07'
validStartDate = '2013-08-08'
validEndDate = '2014-12-31'
testStartDate = '2015-01-02'
testEndDate = '2016-07-01'
train = df[(df[names[0]]>=trainStartDate) & (df[names[0]]<=trainEndDate)]
valid = df[(df[names[0]]>=validStartDate) & (df[names[0]]<=validEndDate)]
test = df[(df[names[0]]>=testStartDate) & (df[names[0]]<=testEndDate)]

In [6]:
## Converting train into train_x and train_y 
trainX = train.iloc[:,2:]
trainY = train.iloc[:,1]
validX = valid.iloc[:,2:]
validY = valid.iloc[:,1]
testX = test.iloc[:,2:]
testY = test.iloc[:,1]
print(trainY.value_counts())
# 1    873
# 0    738
# delete all the other variables
del(train,valid,test,trainStartDate,trainEndDate,validStartDate,validEndDate,testStartDate,testEndDate)

1    679
0    579
Name: Label, dtype: int64


In [7]:
def join_reviews(df):
# Function to join multiple columns in each row to form corpus for each day
    joinedReviews = df.apply(lambda x: ''.join(str(x.values)), axis=1)
    return joinedReviews
## Joined the reviews together for each day
## Should devise a mechansim to weight the reviews
joinedTrainX = join_reviews(trainX)
joinedValidX = join_reviews(validX)
joinedTestX = join_reviews(testX)

In [8]:
def pre_process(text):
# Basic form of pre-processing. Mainly removing 'b' character 
# representing bytes till 467 row.   
# Also remove punctuations and convert to lower case
	process_text=text.replace('\n','').replace('"b','').replace("'b",'')
	for punc in list(punctuation):
		process_text=process_text.replace(punc,'').lower()
	process_text = process_text.strip()
	# Remove extra spaces
	process_text=re.sub(' +',' ',process_text)
	return process_text
joinedTrainX = joinedTrainX.apply(lambda x: pre_process(x))
joinedValidX = joinedValidX.apply(lambda x: pre_process(x))
joinedTestX = joinedTestX.apply(lambda x: pre_process(x))

In [9]:
# Keras tokenizer and sequence padder 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# Define max top words in vocabulary
top_words = 10000
tokenizer = Tokenizer(nb_words=top_words)
tokenizer.fit_on_texts(joinedTrainX)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.
/Users/lin/anaconda/lib/python3.6/site-packages/keras/preprocessing/text.py:145: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 33580 unique tokens.


In [10]:
## Fit tokenizer
def generate_sequence(text,MAX_SEQUENCE_LENGTH):
	sequence= tokenizer.texts_to_sequences(text)
	sequence = pad_sequences(sequence, maxlen=MAX_SEQUENCE_LENGTH)
	return sequence
## Maximum sequence length
max_words = 400
sequencesTrainX = generate_sequence(joinedTrainX,max_words)
sequencesValidX = generate_sequence(joinedValidX,max_words)
sequencesTestX = generate_sequence(joinedTestX,max_words)

In [12]:
## Building the model
## First we will use word embedding and then LSTM layer over it 
from keras.models import Sequential
from keras.layers import Dense,LSTM,Flatten
from keras.layers.embeddings import Embedding
# Seed for reproducability
seed = 1
np.random.seed(seed)
## Dimension of embedding layer
embedding_dim = 128
model = Sequential()
model.add(Embedding(top_words, embedding_dim, input_length=max_words,dropout=0.2))
# model.add(Convolution1D(nb_filter=32, filter_length=3, border_mode='same', activation='relu'))
# model.add(MaxPooling1D(pool_length=2))
model.add(LSTM(128,dropout_W=0.2,dropout_U=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model.summary())
batch_size = 32
## Fit the model
model.fit(sequencesTrainX, trainY, batch_size=batch_size, nb_epoch=5,
          validation_data=(sequencesValidX, validY),verbose=1)
# Ideally should have separate validation data          
score, acc = model.evaluate(sequencesTestX, testY,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

# Ideally should use grid search 
# Hyperparameters
# Vocabulary = 5000
# Max sequence length = 500
# Embedding dimension = 128
# Embedding dropout = 0.2
# Lstm Neurons = 128
# Epoch =5

# loss: 0.6653 - acc: 0.6102 - val_loss: 0.6924 - val_acc: 0.5132
# Test score: 0.692432944106
# Test accuracy: 0.513227514804

# Vocabulary = 10000
# loss: 0.4408 - acc: 0.8535 - val_loss: 0.7931 - val_acc: 0.5344
# Test score: 0.793134343056
# Test accuracy: 0.534391534549

# Vocabulary = 20000

# loss: 0.4714 - acc: 0.8597 - val_loss: 0.7335 - val_acc: 0.5106
# Test score: 0.733532233528
# Test accuracy: 0.510582010267

/Users/lin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  if sys.path[0] == '':
/Users/lin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, dropout=0.2, recurrent_dropout=0.2)`
  from ipykernel import kernelapp as app
/Users/lin/anaconda/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 1258 samples, validate on 353 samples
Epoch 1/5
1258/1258 [==============================] - 92s - loss: 0.6912 - acc: 0.5294 - val_loss: 0.6880 - val_acc: 0.5496
Epoch 2/5
1258/1258 [==============================] - 90s - loss: 0.6475 - acc: 0.6510 - val_loss: 0.7441 - val_acc: 0.5014
Epoch 3/5
1258/1258 [==============================] - 88s - loss: 0.3333 - acc: 0.8887 - val_loss: 0.9050 - val_acc: 0.5382
Epoch 4/5
1258/1258 [==============================] - 89s - loss: 0.1086 - acc: 0.9690 - val_loss: 1.0824 - val_acc: 0.5184
Epoch 5/5
378/378 [==============================] - 4s     
Test score: 1.58359103796
Test accuracy: 0.505291005291
